# Reconhecedor de Sirenes

### Equipe:

- João Marcos Alcântara Vanderley (jmav)
- Maria Luísa dos Santos Silva (mlss)
- Maria Vitória Soares Muniz (mvsm3)

## Objetivo e Motivação

O objetivo principal deste modelo é reconhecer sons de sirenes, o que é fundamental na detecção de veículos de emergência. Um detector de sirenes pode ser utilizado no contexto de carros autônomos, identificando os sons e tomando uma decisão rápida para dar espaço para ambulâncias, carros de bombeiro, entre outros. Além disso, pode ser utilizado para emitir algum alerta visual para motoristas com deficiência auditiva para que alguma atitude seja tomada.

In [1]:
import os
import librosa
import numpy as np
from scipy.io import wavfile
from scipy.signal import spectrogram
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, recall_score, precision_score
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB


## Base de dados

A base de dados escolhida possui áudios recolhidos de vídeos do youtube, os quais estão divididos em três diferentes pastas: 
- "traffic", que contém sons de trânsito
- "ambulance", que contém sons de ambulância
- "firetruck", que contém sons de carros de bombeiro

Apesar dos dados estarem divididos em duas categorias diferentes nas pastas de origem, os sons de ambulância e carros de bombeiro farão parte da mesma classe no momento da classificação, que representará a classe dos veículos de emergência. Essa escolha foi feita, pois o objetivo principal é a detecção de sirenes em geral, independentemente do veículo que a emitiu.

In [2]:
def read_audio_files(audio_folder):
    audios = []

    # representa as classes de forma bninária, onde "traffic" é 0 e "ambulance" e "firetruck" são 1
    folder_label = {"traffic": 0, "ambulance": 1, "firetruck": 1}

    for label in ["traffic", "ambulance", "firetruck"]:
        label_folder = os.path.join(audio_folder, label)
        for file_name in os.listdir(label_folder):
            file_path = os.path.join(label_folder, file_name)

            if file_path.endswith(".wav"):
                audio_data, sample_rate = librosa.load(file_path, sr=None)
                audios.append([audio_data, sample_rate, folder_label[label]])
                
    return audios


## Extração de Características

Após a leitura dos dados, é realizada a extração das seguintes características:

- Energia Média: é uma medida importante que pode ajudar a distinguir entre sons de baixa e alta intensidade. Sirenes tendem a ter picos de alta energia, enquanto os sons de trânsito são mais constantes em termos de energia.

- Zero Crossing Rate: a taxa de cruzamentos por zero é uma medida que indica a quantidade de vezes que o sinal muda de polaridade. Sinais de sirenes tendem a ter taxas mais altas de cruzamentos por zero devido à natureza oscilatória das sirenes, enquanto os sons de trânsito tendem a ter taxas mais baixas.

- Média Espectral: o espectro é obtido a partir de múltiplas aplicações da FFT ao sinal. A média do espectro de frequência pode ajudar a distinguir entre os dois tipos de áudio, pois os sons de sirenes e de trânsito se comportam de formas diferentes na frequência.

In [3]:
def extract_features(audios, extract_energy_mean=True, extract_zcr=True, extract_spec_mean=True):
    features = []
    labels = []

    for [audio_data, sample_rate, label] in audios:
        energy_mean = np.mean(audio_data**2)

        zero_crossings = librosa.zero_crossings(audio_data, pad=False)
        zero_crossing_rate = np.mean(zero_crossings)

        _, _, spec = spectrogram(audio_data, fs=sample_rate)
        spec_mean = np.mean(spec)
        
        features_list = []
        if extract_energy_mean:
            features_list.append(energy_mean)
        if extract_zcr:
            features_list.append(zero_crossing_rate)
        if extract_spec_mean:
            features_list.append(spec_mean)
            
        features.append(features_list)
        labels.append(label)

    return features, labels

## Avaliação do Modelo

As características extraídas e suas respectivas classificações são aplicadas em um modelo, o qual é avaliado em 30 iterações utilizando validação cruzada estratificada com 10 folds. Esse método é utilizado para garantir que porções diferentes com proporções representativas de cada classe estejam sendo utilizadas no treino e no teste, evitando problemas como overfitting em classes menos representadas.

A base de dados possui o dobro de amostras rotuladas como veículos de emergência em relação ao número de amostras rotuladas como sons de trânsito. Por isso, foi utilizado o SMOTE para gerar amostras sintéticas de sons de trânsito e equilibrar as distribuições das classes no conjunto de treinamento do modelo, auxiliando seu aprendizado sobre as características da classe minoritária ("traffic").

Por fim, para cada iteração de treino e teste, foram calculadas métricas de avaliação, incluindo acurácia, recall e precisão. A acurácia mede a proporção de exemplos classificados corretamente, o recall mede a proporção de exemplos positivos corretamente identificados e a precisão mede a proporção de exemplos positivos identificados corretamente em relação a todos os exemplos identificados como positivos.


In [4]:
def evaluate(model, features, labels):
    cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

    results = {'accuracy': [], 'recall': [], 'precision': []}

    for _ in range(30):
        for train_index, test_index in cv.split(features, labels):
            X_train, X_test = [features[i] for i in train_index], [features[i] for i in test_index]
            y_train, y_test = [labels[i] for i in train_index], [labels[i] for i in test_index]

            smote = SMOTE(sampling_strategy=1.0,random_state=42)
            X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

            model.fit(X_train_smote, y_train_smote)

            y_pred = model.predict(X_test)

            accuracy = accuracy_score(y_test, y_pred)
            recall = recall_score(y_test, y_pred)
            precision = precision_score(y_test, y_pred)

            results['accuracy'].append(accuracy)
            results['recall'].append(recall)
            results['precision'].append(precision)
    
    return results
    

In [5]:
def get_metrics(results):
    mean_accuracy = np.mean(results['accuracy'])
    std_accuracy = np.std(results['accuracy'])

    mean_recall = np.mean(results['recall'])
    std_recall = np.std(results['recall'])

    mean_precision = np.mean(results['precision'])
    std_precision = np.std(results['precision'])

    print(f'Acurácia média: {mean_accuracy:.2f} +/- {std_accuracy:.2f}')
    print(f'Recall médio: {mean_recall:.2f} +/- {std_recall:.2f}')
    print(f'Precisão média: {mean_precision:.2f} +/- {std_precision:.2f}')

In [6]:
audios = read_audio_files("sounds")
features, labels = extract_features(audios)

## Modelos Utilizados

Os modelos escolhidos para serem avaliados foram GaussianNB (Naive Bayes Gaussiano) e Random Forest, para testar o desempenho de um modelo mais simples em comparação com um mais robusto diante das características extraídas da base de dados.

O GaussianNB é um modelo simples e rápido, que assume a independência entre as características dos dados para a classificação. Já o Random Forest é um modelo mais robusto, que lida bem com dados desbalanceados e possui melhor eficiência contra overfitting.

In [7]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
results = evaluate(model, features, labels)
get_metrics(results)

Acurácia média: 0.99 +/- 0.01
Recall médio: 0.99 +/- 0.01
Precisão média: 1.00 +/- 0.00


In [8]:
model = GaussianNB()
results = evaluate(model, features, labels)
get_metrics(results)

Acurácia média: 0.97 +/- 0.02
Recall médio: 0.98 +/- 0.02
Precisão média: 0.98 +/- 0.03


## Conclusão

Ambos modelos obtiveram excelente desempenho, o que se deve à boa escolha de características a serem extraídas dos áudios. Foi percebido que, quando uma das características é removida, os modelos não performam tão bem, como pode ser visto abaixo, com a remoção do Zero Crossing Rates

In [9]:
features, labels = extract_features(audios, extract_zcr=False)

In [10]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
results = evaluate(model, features, labels)
get_metrics(results)

Acurácia média: 0.90 +/- 0.04
Recall médio: 0.86 +/- 0.05
Precisão média: 0.98 +/- 0.03


In [11]:
model = GaussianNB()
results = evaluate(model, features, labels)
get_metrics(results)

Acurácia média: 0.83 +/- 0.04
Recall médio: 0.77 +/- 0.05
Precisão média: 0.98 +/- 0.03
